<a href="https://colab.research.google.com/github/avishaiasaf/SEC-EDGAR-Export-Statements/blob/main/SEC_EDGAR_Export_Statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sec-cik-mapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import urllib.request
import urllib3
from urllib.request import Request, urlopen
import requests
import re
from functools import reduce
from sec_cik_mapper import StockMapper
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
mapper = StockMapper()

filing_parameters = {
      'action': 'getcompany',
      'CIK': mapper.ticker_to_cik['AAPL'],
      'type': '10-k',
      'dateb': '',
      'owner': 'exclude',
      'start': '',
      'output': '',
      'count': '100'
}

def build_params(params):
    '''
    Parameters
    ----------
    params : Dictionary
        parameters to get the EDGAR Search Reports for specific company and filling

    Returns
    ----------
    string
        the url attributes to complete the base url 
    '''

    path = '?'
    for key, value in params.items():
        path = path + f'{key}={value}&'
    return path;

def get_report_list_page(filing_parameters):
      '''
      Parameters
      ----------
      filing_parameters : Dictionary
          parameters to get the EDGAR Search Reports for specific company and filling

      Returns
      ----------
      string
          The complete URL to the EDGAR Search Reports
      '''

      baseUrl = 'https://www.sec.gov/cgi-bin/browse-edgar'
      return baseUrl + build_params(filing_parameters);

def get_data(link):
    '''
    Parameters
    ----------
    link : string
        link to a webpage

    Returns
    ----------
    string
        The content of the URL to be processed by BeautifulSoup
    '''

    hdr = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36'}
    req = requests.get(link,headers=hdr)
    content = req.content
    return content


def get_report_link(cik, accession_number, report_number):
    '''
    Parameters
    ----------
    cik : string
        Company Central Index Key
    accession_number : string
        The number to access all of specific filling reports
    report_number : int
        The number to the specific report requested

    Returns
    ----------
    string
        The URL for a specific report from a specific filling
    '''
    if report_number.isnumeric():
        return f'https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/R{report_number}.htm'
    else:
        return report_number

def get_report_list_accession_numbers(url):
    '''
    Parameters
    ----------
    url : string
        A URL to the EDGAR Search Reports for specific company and flling type

    Returns
    ----------
    Array
        The Accession Numbers for each report from the EDGAR Search Reports page
    '''

    u = get_data(url)
    soup = BeautifulSoup(u, 'html.parser')
    accession_number = re.compile(r'accession_number=+(.*).?(?=&)')
    accession_numbers = list(map(lambda x : accession_number.search(str(x)).group(1).replace('-', '') ,soup.find_all(id='interactiveDataBtn')))
    return accession_numbers

def generate_per_statement_links(accession_numbers):
    '''
    Parameters
    ----------
    accession_numbers : Array
        Accession Numbers for a series of company reports with the same filling type but different dates

    Returns
    ----------
    Array
        Links for the Balance Sheet reports for all filling dates
    Array
        Links for the Income Statement reports for all filling dates
    Array
        Links for the Cash Flow reports for all filling dates
    '''

    balance_sheets = []
    income_statements = []
    cashflow_statements = []
    for number in accession_numbers:
        interactive_data_page = get_interactive_data_page(filing_parameters['CIK'], number)
        print('Interactive Data Page ', interactive_data_page)
        page_elements = get_interactive_data_content(interactive_data_page)
        b = get_interactive_data_links(page_elements, 'Consolidated Balance Sheets')
        c = get_interactive_data_links(page_elements, 'Consolidated Statements of Cash Flows')
        i = get_interactive_data_links(page_elements, 'Consolidated Statements Of Operations')

        balance_sheets.append(get_report_link(filing_parameters['CIK'], number, b))
        income_statements.append(get_report_link(filing_parameters['CIK'], number, i))
        cashflow_statements.append(get_report_link(filing_parameters['CIK'], number, c))
    return balance_sheets, income_statements, cashflow_statements

def get_interactive_data_page(cik, accession_number):
    '''
    Parameters
    ----------
    cik : string
        Company Central Index Key
    accession_number
        The number to access all of specific filling reports

    Returns
    ----------
    string
        The URL to the Interactive Data page that contains all financial statements
    '''

    return f'https://www.sec.gov/cgi-bin/viewer?action=view&cik={cik}&accession_number={accession_number}&xbrl_type=v'

def get_interactive_data_content(page_link):
    '''
    Parameters
    ----------
    page_link : string
        Company Central Index Key

    Returns
    ----------
    Array
        All the A elements from the Interactive Data page that should be investigated to extract the financial reports links
    '''

    u = get_data(page_link)
    soup = BeautifulSoup(u, 'html.parser')
    financial_statements_tab = soup.find('a', string='Financial Statements').parent
    return financial_statements_tab.select('li > a')
    # return soup.find_all('a')

def get_interactive_data_links(page_elements, report_description):
    '''
    Parameters
    ----------
    page_elements : Array
        The elements returned from the Interactive Data page
    report_description : string
        The phrase that shall indicate a specific financial statement

    Returns
    ----------
    int
        The number for that specific financial report
    '''

    try:
        marker=0
        for i,j in enumerate(page_elements):
            # turn each table into a string
            str = j.text
            if str.find(report_description) != -1:
                marker = i
        function_number = re.compile(r'javascript:loadReport\(([^\)(]+)\)')
        return function_number.search(page_elements[marker].prettify()).group(1)
    except:
        return 'Element_Not_Found'

def process_financial_statement(url):
    '''
    Parameters
    ----------
    url : string
        The URL to the specific financial statement to extract

    Returns
    ----------
    pandas DataFrame
        The financial report from the URL
    '''

    u = get_data(url)
    soup = BeautifulSoup(u, 'html.parser')
    tables = soup.find_all('table', class_="report")
    if len(tables) == 0:
        return

    marker=0
    for i,j in enumerate(tables):
        # turn each table into a string
        str = j.text
        # find this tile in part of the string
        if str.find('asset') != -1:
            marker = i

    marker
    d = []

    for i,j in enumerate(tables[marker].find_all('tr')):
        for b,c in enumerate(j.find_all(['td', 'th'])):
            if c.text.find('Months Ended') == -1:
                d.append({'Row' : i, 'Cell': b, 'Balance': c.text.strip()})

    df= pd.DataFrame(d)

    df1 = df["Balance"].str.replace("\n","")
    df1 = df["Balance"].str.replace('\\n',' ', regex=True)
    df1 = df["Balance"].str.replace("$","")

    number_of_years = df['Row'].value_counts().max()
    # print(number_of_years)
    headers = np.array(df['Balance'].iloc[:number_of_years].T)
    df = pd.DataFrame([df['Row'], df['Cell'], df1])


    def connect_line(df, headers, new_line):
      df1 = pd.DataFrame([headers, new_line])
      return df.append(df1.iloc[1:], ignore_index=True)

    df = df.T

    df1 = pd.DataFrame()

    for i in range(0, df.shape[0], number_of_years):
        df1 = connect_line(df1, np.array(df['Balance'].iloc[:number_of_years].T), df['Balance'].iloc[i:i+number_of_years].T)

    columns = []
    for col in range(1, number_of_years):
        columns.append(df1[col].str.replace('\\n',  ' ', regex=True))
    # col_1 = df1[1].str.replace('\\n',  ' ', regex=True)
    # col_2 = df1[2].str.replace('\\n',  ' ', regex=True)
    df1 = pd.DataFrame([df1[0], *columns])

    return replace_dataframe_headers(df1.T)

def get_multiple_statements(links):
    '''
    Parameters
    ----------
    links : Array
        Links to series of financial statements of the same type (ex. Balance Sheets)

    Returns
    ----------
    Array 
        Each report as a pandas DataFrame
    '''

    statements = []
    for link in links:
        if link != 'Element_Not_Found':
            print(link)
            statements.append(process_financial_statement(link))
    return statements

def replace_multiple(chars, str, to_char):
    '''
    Parameters
    ----------
    chars : Array
        Characters to remove from the string
    str : string
        String to modify
    to_char : string
        Characters that will replace the removed ones

    Returns
    ----------
    string 
        The string with all the characters replaced
    '''

    for char in chars:
        str = str.replace(char, to_char)
    return str


def replace_dataframe_headers(df):
    '''
    Parameters
    ----------
    df : pandas DataFrame
        The report to modify column names

    Returns
    ----------
    pandas DataFrame 
        The report with modified column names, minus the first row
    '''

    headers = df.iloc[0:1].values.tolist()
    new_headers = []
    for header in headers[0]:
        header = replace_multiple(['.',' ', ','], header, '_')
        new_headers.append(header)
    df.columns = new_headers
    return df.iloc[1:]


def merge_reports_dataframes(dfs):
    # Need to check merging reports with different units of measure (Thousands/ Millions)
    '''
    Parameters
    ----------
    dfs : Array of pandas DataFrames
        The reports to be merged

    Returns
    ----------
    pandas DataFrame 
        The merged report
    '''
    return reduce(lambda init, current: pd.merge(init, current, how="left"), dfs)

def get_statements_by_ticker(ticker, num_of_years):
    '''
    Parameters
    ----------
    ticker : string
        The ticker of a specific company

    Returns
    ----------
    Array of pandas DataFrame 
        The merged reports for specific company
    '''

    filing_parameters['CIK'] = mapper.ticker_to_cik[ticker],
    report_list_page = get_report_list_page(filing_parameters)
    accession_nembers = get_report_list_accession_numbers(report_list_page)
    b, i, c = generate_per_statement_links(accession_nembers)

    print(i)

    income_statements = get_multiple_statements(i)
    balance_sheets = get_multiple_statements(b)
    cashflows = get_multiple_statements(c)

    print(i)

    consolidated_income_statement = merge_reports_dataframes(income_statements[:num_of_years])
    consolidated_balance_sheet = merge_reports_dataframes(balance_sheets[:num_of_years])
    consolidated_cashflow = merge_reports_dataframes(cashflows[:num_of_years])

    return consolidated_income_statement, consolidated_balance_sheet, consolidated_cashflow



# interactive_data_page = get_interactive_data_page(filing_parameters['CIK'], accession_nembers[0])
# page_elements = get_interactive_data_content(interactive_data_page)
# print(get_interactive_data_links(page_elements, 'Balance'))
# print(get_interactive_data_links(page_elements, 'Cash Flow'))
# print(get_interactive_data_links(page_elements, 'Operation'))
# accession_nembers
# 000101872416000172
# 000101872416000172
# https://www.sec.gov/Archives/edgar/data/1018724/000101872422000019/R2.htm


In [ ]:
i, b, c = get_statements_by_ticker('AAPL', 3)
i

[]
[]


TypeError: ignored

In [ ]:
report_list_page = get_report_list_page(filing_parameters)
accession_nembers = get_report_list_accession_numbers(report_list_page)
b, i, c = generate_per_statement_links(accession_nembers)

# report_list_page
i
# accession_nembers
# process_financial_statement('https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/R3.htm')

Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019321000105&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019320000096&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019319000119&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019318000145&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019317000070&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000162828016020309&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000119312515356351&xbrl_type=v
Interactive Data Page  https://www.sec.gov/cgi-bin/viewer?action=view

['https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000032019320000096/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000032019319000119/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000032019318000145/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000032019317000070/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000162828016020309/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312515356351/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312514383437/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312513416534/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312512444068/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312511282113/R2.htm',
 'https://www.sec.gov/Archives/edgar/data/0000320193/000119312510238044/R2.htm',
 'https://www.sec.gov/Archiv

In [ ]:
# Test
# get_interactive_data_content(get_interactive_data_page(filing_parameters['CIK'], accession_nembers[0]))
get_interactive_data_page(filing_parameters['CIK'], accession_nembers[0])

'https://www.sec.gov/cgi-bin/viewer?action=view&cik=0000320193&accession_number=000032019321000105&xbrl_type=v'

In [ ]:
income_statements = get_multiple_statements(i)
# cahsflows = get_multiple_statements(c)
# balance_sheets = get_multiple_statements(b)

https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/R2.htm


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:242: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


https://www.sec.gov/Archives/edgar/data/0000320193/000032019320000096/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000032019319000119/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000032019318000145/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000032019317000070/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000162828016020309/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312515356351/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312514383437/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312513416534/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312512444068/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312511282113/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312510238044/R2.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312510012091/R1.htm
https://www.sec.gov/Archives/edgar/data/0000320193/000119312509214859/R1.htm

In [ ]:
# cahsflows[0]
final_income_statement = merge_reports_dataframes(income_statements[:3])
# balance_sheets[0]
# b

In [ ]:
final_income_statement['Sep__30__2017']=final_income_statement['Sep__30__2017'].astype(float)
final_income_statement['Sep__29__2018']=final_income_statement['Sep__29__2018'].astype(float)
final_income_statement['Sep__28__2019']=final_income_statement['Sep__28__2019'].astype(float)
final_income_statement['Sep__26__2020']=final_income_statement['Sep__26__2020'].astype(float)
final_income_statement['Sep__25__2021']=final_income_statement['Sep__25__2021'].astype(float)
final_income_statement.dtypes#.iloc[1:3, 3:].plot()

ValueError: ignored

In [ ]:
'''
    R2 - Income Statement
    R4 - Balance Sheet
    R6 - Cashflow Statement
'''

url = 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/R4.htm'
url_msft_income_statement = 'https://www.sec.gov/Archives/edgar/data/789019/000156459021051992/R2.htm'
url_2 = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000032019322000070/aapl-20220625.htm'

request_site = Request(url, headers={"User-Agent": "Mozilla/5.0"})
u = get_data(url_msft_income_statement)
#'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/R1.htm'


#u = urllib.request.urlopen('https://www.sec.gov/Archives/edgar/data/1141807/000114180718000005/0001141807-18-000005.txt', headers={"User-Agent": "Mozilla/5.0"})

soup = BeautifulSoup(u, 'html.parser')

# find all tables in the document, one of which is the Balance Sheet.
soup.find_all('table')
tables = soup.find_all('table', class_="report")
tables

len(tables)
# Per visual inspection of the 10-K, the Balance Sheet has "ASSETS" written in all caps. Find the table or tables that have "ASSETS"
marker=0
for i,j in enumerate(tables):
    # turn each table into a string
    str = j.text
    #print(str)
    # find this tile in part of the string
    if str.find('asset') != -1:
        marker = i

marker

# print(tables[marker].prettify())

# get a count of all td items per row. This will provide a row count, a td count within each row, and the text of the td item.
for i,j in enumerate(tables[marker].find_all('tr')):
    for b,c in enumerate(j.find_all(['td', 'th'])):
        # print(i, b, c.text)
        pass

# Use the above template to create a new dataframe with columns for i, b, and c.text.
d = []

for i,j in enumerate(tables[marker].find_all('tr')):
    for b,c in enumerate(j.find_all(['td', 'th'])):
        if c.text.find('Months Ended') == -1:
            # print(c.text)
            d.append({'Row' : i, 'Cell': b, 'Balance': c.text})

d
df= pd.DataFrame(d)

df

# # Go through and clean out td items that are throwing the table off. The goal is to have each financial statement line item followed by each of the two years on the balance sheet.
# df= df.drop(df[df['Balance'] == "$"].index)
# # df= df.drop(df[df['Balance'] == ")"].index)
# # df= df.drop(df[df['Balance'] == "("].index)
# df= df.drop(df[df['Balance'] == ""].index)
# df= df.drop(df[df['Balance'] == "\n"].index)
df1 = df["Balance"].str.replace("\n","")
df1 = df["Balance"].str.replace('\\n',' ', regex=True)
df1 = df["Balance"].str.replace("$","")

#df1

# number_of_columns = df.groupby(['Row'])['Row'].count().max()
# print(number_of_columns)
number_of_years = df['Row'].value_counts().max()
headers = np.array(df['Balance'].iloc[:4].T)
df = pd.DataFrame([df['Row'], df['Cell'], df1])

df.T

# # The above is closer to the desired result, but there still appear to be rows with null values. Manually display what appear to be missing rows.
# # df['Balance'][9]


def connect_line(df, headers, new_line):
  df1 = pd.DataFrame([headers, new_line])
  return df.append(df1.iloc[1:], ignore_index=True)



df = df.T

# df['Balance']

df1 = pd.DataFrame()

for i in range(0, df.shape[0], number_of_years):
    df1 = connect_line(df1, np.array(df['Balance'].iloc[:number_of_years].T), df['Balance'].iloc[i:i+number_of_years].T)

col_1 = df1[1].str.replace('\\n',  ' ', regex=True)
col_2 = df1[2].str.replace('\\n',  ' ', regex=True)
df1 = pd.DataFrame([df1[0], col_1, col_2])
df1.T
# col

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,0,1,2
0,"INCOME STATEMENTS - USD () shares in Millions,...","Sep. 30, 2021","Sep. 30, 2020"
1,Revenue,"45,317","37,154"
2,Cost of revenue,"13,646","11,002"
3,Gross margin,"31,671","26,152"
4,Research and development,"5,599","4,926"
5,Sales and marketing,"4,547","4,231"
6,General and administrative,"1,287","1,119"
7,Operating income,"20,238","15,876"
8,"Other income, net",286,248
9,Income before income taxes,"20,524","16,124"
